*version 190910.0*

# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment.

In addition, I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

## Notes

1. Partial marks may be awarded if the answer is incorrect, or removed even if the answer is correct! Within each function show your understanding of course concepts such as regex, `groupby()`, `apply()`, etc. as appropriate to the question. If you unnecessarily hand code data in your functions instead of using pandas cleaning mechanisms you may be docked grades.
2. Do not including data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
3. I highly suggest that you first tackle the correlation questions, as they are all similar and worth the majority of grades for this assignment. This is by design!
4. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
5. This assignment **is not autograded**. Go to the course shell and upload your .ipynb file there as assignment4.ipynb.
6. You can earn up to 100% on this assignment by creating correlations for a single year (2016) for which there is population data. You can earn up to 105% on this assignment by going further, and creating correlations for multiple years (e.g. across the years of data we have on the sports), but it's expected that there would be a short discussion of the limitations of your approach as far as interpretation goes. Finally, if you find new population data that covers a broader set of years and integrate it into a solution across these years and sports and provide that interpretation you can earn up to 110% on the assignment.

<h1> Assumptions and Callouts </h1>

<ul> 
    <li>WL ratio is calculated Wins / Losses but can be changed in a flag</li>
    <li>Had to re label the year values for 2017 values for NHL from the cities tables index positoins 69:102, this is hardcoded</li>
    <li>If I miss some of the teams its becuase the table is dirty but I came up with some sophisticated regex an pandas functions to handle edge cases using pandas.Series.str functions.</li>
    <li>To get all the values I needed I had to store some indexes in weird places but overall I am happy with the solution and how flexible it is</li>
    <li>The amount of functions is because I wanted to break out tasks and to make things flexible. There is a lot of calling functions within functions that I think will be poor arhitecture for production</li>
    <li><b>There is extracredit output for each, You can run each method with a different year or range of years</b></li>
</ul>


<h1> Description of Whats Below </h1>
<ol>
    <li>Primary function set that gets used</li>
    <li>Answer 1 synopsis</li>
    <li>Answer 1 code cell</li>
    <li>Answer 1 output</li>
    <li>Answer 2 Synopsis</li>
    <li>Answer 2 code cell</li>
    <li>Answer 2 output</li>
    <li>Answer 2 analysis</li>
    <li>Fin!</li>
</ol>






In [12]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re
from IPython.display import display


sports = ['nfl', 'nhl', 'nba', 'mlb']
sport_cols = ['year', 'team', 'W', 'L']

#set ratio calculation
types = ['W/L', 'W/(W+L)']
#set ratio to 'W/L' change to types[1] for 'W/(W+L)'
wlratio = types[0]


# helper functions over here
def fix_stupid_year(df):
    df.loc[69:102, 'year'] = 2016
    return df

def read_cities():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    return cities

def read_sport(sport):
    assert sport in sports, f'Cannot read file invalid sport: {sport}'
    if sport == 'mlb':
        mlb_df=pd.read_csv("assets/mlb.csv", usecols=sport_cols)
        return mlb_df
    if sport == 'nhl':
        nhl_df = pd.read_csv("assets/nhl.csv", usecols=sport_cols)
        nhl_df = fix_stupid_year(nhl_df)
        return nhl_df
    if sport == 'nba':
        nba_df=pd.read_csv("assets/nba.csv", usecols=sport_cols)
        return nba_df
    if sport == 'nfl':
        nfl_df=pd.read_csv("assets/nfl.csv", usecols=sport_cols)
        return nfl_df

def read_cities():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    return cities

def calculate_win_loss(df, sport):
    if wlratio == 'W/(W+L)':
        df[f'{sport}-WL%'] = df['W'].astype(float)/(df['W'].astype(float)+df['L'].astype(float))
    if wlratio == 'W/L':
        df[f'{sport}-WL%'] = df['W'].astype(float)/df['L'].astype(float)
    return df

#gives you the metro and teams
def get_team_cities_pop_by_sport(df, sportu, sport):
    df[sportu] = df[sportu].str.replace(' ', '_')
    def repl(m):
        if '_' in m[0]:
            return re.sub(r"([A-Z][a-z]+)([A-Z])", r"\1 \2", m[0])
        else:
            return re.sub(r"([A-Z]\w+)([A-Z])", r"\1 \2", m[0])
    df[sportu] = df[sportu].str.replace('\w+', repl)
    df = df[sportu].str.extractall(f'(?P<{sport}>\w+)')
    df[sport] = df[sport].str.replace('_', ' ')
    return df

# cities lookup and read
# modify the df to support the ripper above
def get_cityPopTeam(sport=''):
    assert sport in sports, 'Not a valid sport'
    # specify some vars
    cols = ['Metropolitan area', 'Population', f'{sport}']
    df = read_cities()
    su = sport.upper()
    # set index for returning some code
    df = df.set_index(['Metropolitan area','Population (2016 est.)[8]'])
    # remove brackets
    df[su] = df[su].str.replace('\[.*\]', '')
    # run function to get teams and population and metro
    df = get_team_cities_pop_by_sport(df, su, sport)
    df = df.reset_index().drop(columns = 'match')
    df.columns = cols
    return df

# win loss values for each sport and year
def get_WinLossPerc(df, year, sport):
    teams = get_cityPopTeam(sport)[sport]
    df = df[df['W'].astype(str).str.contains('[0-9\.]')]
    df = df[df['L'].astype(str).str.contains('[0-9\.]')]
    df['team'] = df['team'].str.strip('+*').str.replace('\(\d*\)','').str.replace('[*]', '')
    # silly edge case handling here
    def repl(m):
        return m.group('team')
    matches = '|'.join(teams)
    df[sport] = df['team'].str.replace(f'(?P<trash>[\w+\s+.])+(?P<team>{matches})', repl).str.lstrip().str.rstrip()
    # can we do multiple years in one request ?
    if type(year) == int:
        df = df[df['year']==year]
    if type(year) == list:
        df['year'] = df['year'].astype(str)
        years_c = '|'.join([str(y) for y in range(year[0],year[-1]+1)])
        df = df[df['year'].str.contains(f'{years_c}')]
    df = calculate_win_loss(df, sport)
    return df
    
def get_WinLossPerc_by_team_year(year, sport):
    df_sport = read_sport(sport)
    df = get_WinLossPerc(df_sport, year, sport)
    return df

def wl_ratio_by_sport_year(year, sport):
    def join_a_df(df1, df2, sport):
        return pd.merge(df1, df2, on=sport, how='inner')
    df1 = get_WinLossPerc_by_team_year(year, sport).sort_index()
    df2 = get_cityPopTeam(sport).sort_index()
    df = join_a_df(df1,df2, sport)
    return df

def region_wl_ratio(year, sport):
    df = wl_ratio_by_sport_year(year, sport).groupby(['Metropolitan area','Population', 'year']).agg({f'{sport}-WL%': np.mean}).reset_index()
    df['Population'] = df['Population'].astype(float)
    return df

def wl_ratio_by_sport_year_combined(year):
    from functools import reduce
    frames = [region_wl_ratio(year, s) for s in sports]
    df = reduce(lambda left,right: pd.merge(left,right, on=['Metropolitan area','Population', 'year'], how='outer'), frames)
    df = df.drop(columns=['Population', 'year']).set_index('Metropolitan area')
    return df


# display(region_wl_ratio(2016, 'nfl'))
# display(wl_ratio_by_sport_year_combined(2016))


<h1>Answer1</h1>
<p>
    In the below cell we have the answer for each correlation of Win/Loss calculated using formula W/L% = W/L. It can be modified in the code above. In the second cell below you can run the answer1 code with different years or a range of years to estimate the correlation over several year time span. <b>Running with different years assumes that the population was the same for all years and matches 2016 population levels</b>. Output is printed below the cell.

<ul>
    <li>run answer1(2016)</li>
    <li>run answer1(2017)</li>
    <li>run answer1([2015-2017])</li>
</ul>

</p>

In [15]:
def answer1(year):
    def get_region_df(year, sport):
        df = region_wl_ratio(year, sport)
        return (df['Population'], df[f'{sport}-WL%'])
        
    def nhl_correlation(): # worth 20% of the assignment grade
        x,y = get_region_df(year, 'nhl')
        return stats.pearsonr(x, y)

    def nba_correlation(): # worth 20% of the assignment grade
        x,y = get_region_df(year,'nba')
        return stats.pearsonr(x, y)

    def mlb_correlation(): # worth 20% of the assignment grade
        x,y = get_region_df(year,'mlb')
        return stats.pearsonr(x, y)

    def nfl_correlation(): # worth 20% of the assignment grade
        x,y = get_region_df(year,'nfl')
        return stats.pearsonr(x, y)
    
    print(f''' 
    pstat nfl: {nfl_correlation()}
    pstat nba: {nba_correlation()}
    pstat mlb: {mlb_correlation()}
    pstat nhl: {nhl_correlation()}
    ''')
    
print([y for y in range(2015, 2017+1)])
    

[2015, 2016, 2017]


In [16]:
# run this cell to get answer1 for 2016
print('Output for 2016')
answer1(2016)
# 2017
print('Output for 2017')
answer1(2017)
# 2015-2017
print('Output for 2015-2017')
answer1([2015,2017])

Output for 2016
 
    pstat nfl: (0.019580248017213232, 0.9212221061995737)
    pstat nba: (-0.033484833919025916, 0.8656755534205006)
    pstat mlb: (0.24421193385821707, 0.22924420941793794)
    pstat nhl: (0.25254413751070315, 0.20376996015502047)
    
Output for 2017
 
    pstat nfl: (-0.1281086980413031, 0.5159076645692733)
    pstat nba: (-0.06294306113515093, 0.7503378189986912)
    pstat mlb: (0.08882434547042775, 0.6661072083618192)
    pstat nhl: (-0.0823820905079294, 0.682899425247494)
    
Output for 2015-2017
 
    pstat nfl: (-0.09214329147736473, 0.4044902363083118)
    pstat nba: (-0.02093171209750787, 0.850101866713202)
    pstat mlb: (0.17411941061629854, 0.1273525527747347)
    pstat nhl: (0.03867004701043075, 0.7317870778096578)
    


<h1>Answer 2</h1>
<p>
    This is to calculate the correlation of sport performance relative to region for two sports. I first get all the WL ratios in a df and then extract two columns at a time by name into a new df and drop the values that contain nan. I should be left with 6 unique combinations of 2 sports and metro areas. Dropping rows that are NaN leaves me with only metro areas that have those two sports. Then I run my ttest and update my 4x4 matrix. The value is visbile below in a print out. Run the cells below to see the output.
    <ul>
        <li>run answer2(2016)</li>
        <li>run answer2(2017)</li>
        <li>run answer2([2015-2017])</li>
    </ul>
<p>

In [17]:
def answer2(year):
    from itertools import combinations as c
    df = wl_ratio_by_sport_year_combined(year)
    cols = ["NFL","NBA","NHL","MLB"]
    p_values=pd.DataFrame({k:np.nan for (k) in cols},index=cols)       
    #simple naive way to rename the columns
    df.columns = cols
    #create list of series and combinations
    for c in list(c([l for l,s in df.iteritems()],2)):
        df_ = df[[s for s in c]].dropna()
        s, p = stats.ttest_ind(df_[c[0]],df_[c[1]])
        p_values.loc[c[0],c[1]]=p
        p_values.loc[c[1],c[0]]=p 
    return display(p_values)

In [18]:
answer2(2016)
answer2(2017)
answer2([2015,2017])

,NFL,NBA,NHL,MLB
NFL,NaN,0.778587,0.229729,0.073792
NBA,0.778587,NaN,0.937503,0.001752
NHL,0.229729,0.937503,NaN,0.358145
MLB,0.073792,0.001752,0.358145,NaN


,NFL,NBA,NHL,MLB
NFL,NaN,0.358955,0.147295,0.142817
NBA,0.358955,NaN,0.677857,0.027587
NHL,0.147295,0.677857,NaN,0.507876
MLB,0.142817,0.027587,0.507876,NaN


,NFL,NBA,NHL,MLB
NFL,NaN,0.463677,0.039569,0.003454
NBA,0.463677,NaN,0.659874,0.000004
NHL,0.039569,0.659874,NaN,0.087137
MLB,0.003454,0.000004,0.087137,NaN


<h3>Answer 2 analysis</h3>

<p>
    Null hypothesis is that sports teams perform the same relative to each other between regions. This is an indepedent test of two samples. So we are comparing the averages to see if the sports that exist in the same regions actually perfrom the same within a given alpha threshold. wl_avg(sport1) = wl_avg(sport2). If the pvalue reported by ttest_ind function is less than our alpha threshold than we reject the null hypothesis and say the the sports perform differently relatively to each other in the regions where in whic they both occupy. This is becoming lengthy.
    
<h4>IF</h4>
<b>H0; mu1=mu2 &</b>
<b>HA; mu1!=mu2 </b>
<h4>AND</h4>
<b>alpha = 0.05</b>
<h4>WHERE</h4>
<b>mu1 = avg sport1 &</b>
<b>mu2 = avg sport2</b>
</p>
<p> For 2016 values for each sport combination </p>

<ul>
    <li>('NFL', 'NHL'), accept</li>
    <li>('NFL', 'NBA'), accept</li>
    <li>('NFL', 'MLB'), accept</li>
    <li>('NHL', 'NBA'), accept</li>
    <li>('NHL', 'MLB'), accept</li>
    <li>('NBA', 'MLB'), reject</li>
</ul>

<b>It appears that (NBA and MLB) teams differ in their W/L ratio across the regions analyzed.<b>

